In [ ]:
import os
import numpy as np
import open3d as o3d
import pandas as pd
from scipy.spatial import ConvexHull
from scipy.spatial.distance import pdist
from skimage.measure import EllipseModel
import matplotlib.pyplot as plt

In [ ]:
import open3d as o3d

# Carrega nuvem de pontos
pcd = arquivo

# Cria um grid de voxels (resolução definida pelo tamanho do voxel)
voxel_size = 0.01  # Ex: 1 cm
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size)

# Número de voxels preenchidos
num_voxels = len(voxel_grid.get_voxels())

# Volume total estimado
voxel_volume = voxel_size ** 3
total_volume = num_voxels * voxel_volume
return voxel_volume


In [ ]:
import open3d as o3d
# Carrega nuvem
pcd = o3d.io.read_point_cloud(arquivo)

# Estima normais, necessário para reconstrução
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.06, max_nn=30))

# Reconstrói a malha
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=5)


# Calcula área da malha
mesh_area = mesh.get_surface_area()

print(f"Área superficial estimada: {mesh_area:.6f} m²")


Área superficial estimada: 0.007599 m²


In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.interpolate import interp1d
from scipy.interpolate import interp1d
from scipy.integrate import simps
df_new = pd.DataFrame(
)

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)
    return inlier_cloud
    # print("Showing outliers (red) and inliers (gray): ")
    # outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    # o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

# Lista de arquivos .ply
arquivos_ovos = os.listdir(r'C:\Users\mhmon\ovos-pv\ovos3')

# Carregar dados reais
df = pd.read_csv(r'C:\Users\mhmon\ovos-pv\Treino_V4.csv')

features = {}
# Loop para testar cada ovo
for i, row in df.iterrows():


    nome = row['nome']
    # nome = 'ovo'+str(num)
    arquivo = f'C:\\Users\\mhmon\\ovos-pv\\ovos3\\{nome}'
    numero_ovo = nome.replace('ovo', '')  # Extrai o número do ovo
    numero_ovo = int(numero_ovo.replace('.ply', ''))  # Extrai o número do ovo
    print(numero_ovo)
    if (not(numero_ovo <= 1000)):  # Filtra os ovos dentro do intervalo desejado
        continue

    if not os.path.exists(arquivo):  # Verifica se o arquivo existe
        print(f"Arquivo {arquivo} não encontrado, pulando...")
        continue
    print(arquivo)
    # Carregar nuvem de pontos
    pcd = o3d.io.read_point_cloud(arquivo)
    # Carrega nuvem de pontos

    # Cria um grid de voxels (resolução definida pelo tamanho do voxel)
    voxel_size = 0.01  # Ex: 1 cm
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size)

    # Número de voxels preenchidos
    num_voxels = len(voxel_grid.get_voxels())

    # Volume total estimado
    voxel_volume = voxel_size ** 3
    total_volume = num_voxels * voxel_volume
    print(num_voxels)


    # Estima normais, necessário para reconstrução
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.06, max_nn=30))

    # Reconstrói a malha
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=5)


    # Calcula área da malha
    mesh_area = mesh.get_surface_area() *10000


    nova_linha = pd.DataFrame([{
        'nome': nome,
        "num_voxel": num_voxels,
        "voxel_area": mesh_area,
    }])


    df_new = pd.concat([df_new, nova_linha], ignore_index=True)



In [ ]:
df = pd.read_csv('Treino_V4.csv')
df2 = df.merge(df_new, how='left',on='nome')

In [ ]:
df2

,nome,L_estimado,L,B,B_estimado,linha,S_OVO,P_OVO,D/4,VOL_OVO,...,area_proj_2d,shape_compactness,CL_PESO,CLA_I_FOR,sup_rev,sup_rev_escala,diam_4,curvatura_media,num_voxel,voxel_area
0,ovo454.ply,59.71,60.70,46.71,46.40,3,68.10,70.5,41.71,69.08,...,15.65,56.48,Jumbo,Redondo,75.18,83.66,28.08,28.68,54,77.592490
1,ovo450.ply,59.24,59.15,45.21,46.30,2,64.73,58.1,37.87,63.39,...,15.44,56.30,Extra,Redondo,75.27,83.76,24.96,25.93,51,60.820990
2,ovo870.ply,62.62,60.53,45.55,45.65,1,66.24,67.4,40.49,65.77,...,15.80,55.26,Extra,Padrão,78.27,87.10,21.84,27.08,46,53.430686
3,ovo39.ply,60.51,58.99,44.60,45.21,3,63.72,65.6,37.87,61.65,...,15.27,55.81,Extra,Padrão,73.06,81.30,26.00,29.47,45,70.146046
4,ovo660.ply,57.56,57.47,44.98,44.76,3,63.15,63.4,38.35,61.01,...,15.41,59.83,Extra,Redondo,73.69,80.47,21.84,29.87,50,68.416709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,ovo80.ply,59.58,58.30,42.91,43.47,2,60.76,59.4,37.79,56.73,...,15.11,58.34,Extra,Padrão,71.64,79.72,20.80,24.85,43,56.140533
679,ovo909.ply,59.67,58.80,43.82,43.57,1,62.44,60.1,39.26,59.48,...,15.20,58.48,Extra,Padrão,71.99,80.11,18.72,26.58,52,46.760735
680,ovo71.ply,57.61,58.90,44.27,45.08,2,63.17,63.7,38.92,60.72,...,15.52,59.75,Extra,Padrão,73.78,82.10,19.76,25.58,46,52.013536
681,ovo604.ply,56.87,57.12,45.92,46.47,1,64.25,64.7,39.45,63.00,...,15.50,58.66,Extra,Redondo,76.03,83.03,19.76,27.26,53,51.823343
